<a href="https://colab.research.google.com/github/podemosaprender/xw1/blob/main/juegos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Juegos como Relaciones Sociales, analizar y diseñar


## ¿Parecidos y diferencias?

Dos hermanitos de 10 y 7 años juegan juegos de mesa. Pelean un poco por cual juego jugar y también cambiando la interpretación de las reglas tratando de no perder. Se acuerdan lo que dijo el otro en partidos anteriores y tratan de usarlo para convencer si les sirve en este partido.

* ¿Se cumple el _reglamento_? ¿Quién decide cómo interpretarlo?
* ¿Siempre hay un _juego por afuera del juego_?
* ¿Se puede **no** jugar a _si querés comer tenes que pagar_?

## Marco de análisis MDA "Mecánicas, Dinámicas, Estéticas"

* ¿Caja linda, juego malo?
* ¿Las piezas son muy chiquitas?
* ¿Es para gente muy inteligente?
* ¿Imposible ganarle a los que juegan hace más que vos?

https://en.wikipedia.org/wiki/MDA_framework



## Mecánicas

* ¿Cuáles son todos los partidos posibles?
* ¿Cuándo ya no podría ganar?
* ¿Requiere velocidad, memoria, cálculo, conocimiento, ...?
* ¿Se puede __enfatizar__ una mecánica para que me favorezca el reglamento?

* ej. Partidas relámpago de ajedrez
* ej. Truco con o sin flor


## Investigar mecánicas con Python

* **Espacio** de partidos posibles
* **Simular** partidos, simular **jugadores** (agentes)
* Descubrir estrategias

### (instalar software, etc.)

In [2]:
!pip install --upgrade plotly >/dev/null 2>&1 ;
;

''

In [24]:
PrepareForWeb= False
PrepareForWeb= True #U: descomentar esta línea antes de guardar en github

In [25]:
import pandas as pd #U: leer y agrupar datos
import numpy as np

import plotly.graph_objects as go #U: graficar con plotly
import plotly.express as px #U: graficar con plotly
import plotly #SEE: https://medium.com/@282abhishek/using-plotly-with-nbconvert-in-google-colab-96834c4f2850
if PrepareForWeb:
  plotly.offline.init_notebook_mode(connected=True)
else:
  plotly.io.renderers.default = 'colab'

## Espacio de partidos posibles

Se pueden listar todos los partidos posibles en algunos casos:

* Tirar una moneda: sale cara o número
* Tirar una moneda dos veces: cc,cn,nc,nn



### Ta-Te-Ti

Puedo aprovechar que hay 9 casilleros y simplemente anotar en la posición correspondiente en que número de jugada se llenaron.

ej.
~~~

_X_  010 000 000
___
___

_X_  010 000 002
___
__O

~~~

¿Cúantos tableros posibles hay? ¿Cuántos de esos son partidos válidos?

¿Cuántos finales para el tablero 123 456 000?

## Jugar es recorrer el espacio de partidos posibles

Me dicen
* "si ahora las cosas estan así"
* "¿qué podría pasar después?"

¿Cómo pasar de las jugadas y dinámicas posibles a "predecir el futuro"?

### Generar una secuencia de jugadas válidas

Como en ajedrez, ej. https://www.chessgames.com/perl/chessgame?gid=1012518

Empecemos con un ejemplo simplísimo, un **Random Walk**

In [26]:
np.random.seed(10)
def deltas(n=100): #U: generar n movimientos en una coordenada
  return np.random.choice([-1,0,1], n)

def walk(n=100): #U: generar un df con un random walk
  df= pd.DataFrame({'dx': deltas(n), 'dy': deltas(n)})
  df['x']= df.dx.cumsum()
  df['y']= df.dy.cumsum()
  return df

df= walk(10)
df.head()

,dx,dy,x,y
0,0,0,0,0
1,0,1,0,1
2,-1,-1,-1,0
3,-1,0,-2,0
4,0,-1,-2,-1


In [27]:
px.scatter(df, x='x', y='y', range_x=[df.x.min(), df.x.max()], range_y=[df.y.min(), df.y.max()], animation_frame=df.index)

## Analizar partidos

### Tramposos en ajedrez

Chess.com echó a un jugador famoso acusándolo de hacer trampa en un [informe estadístico](https://www.chess.com/blog/CHESScom/hans-niemann-report)

The basic concept of cheat detection, particularly at the top level of chess, is both statistical and manual,
involving:

* Comparing the moves made to engine recommended moves
* Removing some moves (opening, some endgame)
* Focusing on key/critical moves
* Discussing with a panel of trained analysts and strong players
* Comparing player past performance and known strength profile
* Comparing a player’s performance to performances of comparable peers
* Looking at the statistical significance of the results (ex. “1 in a million chance of happening
naturally”)
* Looking at if there are behavioral factors at play (ex. “browser behavior”)
* Reviewing time usage when compared to difficulty of the moves on the board

### Backtesting y simulación en finanzas

#### Dólar vs Inflación en Argentina

In [28]:
uva_df_tables= pd.read_html('http://data.podemosaprender.org/finanzas/UVA_BCRA_2022.html')
uva_df= uva_df_tables[0] #A: sabemos que hay una sola!
uva_df.rename(columns={'Valor':'UVA', 'Fecha': 'fecha'}, inplace=True)
uva_df.fecha= pd.to_datetime( uva_df.fecha, format='%Y-%m-%d')

usd_blue_df= pd.read_csv("https://data.podemosaprender.org/finanzas/USD_BLUE_2022.csv", sep="\t")
usd_blue_df.fecha= pd.to_datetime( usd_blue_df.fecha, format='%Y/%m/%d')

uva_usd_df= pd.merge(uva_df, usd_blue_df, on="fecha")

uva_usd_df.head()

,fecha,UVA,compra,venta
0,2022-01-03,97.58,202.0,202.0
1,2022-01-04,97.82,202.5,202.5
2,2022-01-05,97.89,203.5,203.5
3,2022-01-06,97.97,204.5,204.5
4,2022-01-07,98.05,204.0,204.0


In [29]:
fig = px.line(uva_usd_df, x="fecha", y=["UVA","venta"], title='Dólar vs Inflación')
fig.show()

In [30]:
uva_usd_df['usd_en_uva']= uva_usd_df.venta / uva_usd_df.UVA
uva_usd_df.head()

,fecha,UVA,compra,venta,usd_en_uva
0,2022-01-03,97.58,202.0,202.0,2.070096
1,2022-01-04,97.82,202.5,202.5,2.070129
2,2022-01-05,97.89,203.5,203.5,2.078864
3,2022-01-06,97.97,204.5,204.5,2.087374
4,2022-01-07,98.05,204.0,204.0,2.080571


In [31]:
fig = px.line(uva_usd_df, x="fecha", y=["usd_en_uva"], title='USD/UVA')
fig.show()

In [32]:
uva_usd_df['chg']= uva_usd_df.usd_en_uva.pct_change()

In [33]:
px.histogram(uva_usd_df, x='chg', nbins=10, title='Cuantos días para cada % de suba/baja del dólar en uva')

## Juegos como medio para entender

* ¿Cómo sentimos?
* ¿Qué mecánicas nos producen ese sentimiento?
* ¿Cómo podemos cambiar lo que siente y hace cada persona?

Se puede trabajar de eso, casi todos trabajamos de eso sin saberlo.

* Ponemos precios tratando de que no se vayan porque es caro pero cobrar lo más que podamos
* Opt-in vs Opt-out ej [donación de organos](https://www.researchgate.net/figure/Effective-consent-rates-by-country-The-four-leftmost-bars-are-explicit-consent_fig2_7701098)
* https://en.wikipedia.org/wiki/Nudge_theory

![plan muy básico a la izquierda, caro y excesivo a la derecha, el que nos conviene vender en el centro](https://i.pinimg.com/736x/4d/6c/1c/4d6c1c7c5554a78bd3be127cd16a9fe1.jpg)

## "Juegos rotos": nos sentimos defraudados

Muchas veces
* la estética es atractiva o las reglas parecen justas
* nos sentimos defraudados por el resultado

¡Es difícil lograr que un juego NO este roto! Cuando diseñás un juego esta roto casi todo el tiempo.

Por la misma razón muchos "juegos rotos" se usan para estafar: reglas que parecen justas, promesas que te entusiasman, salís defraudado porque apareció una dinámica que no esperaba.

Si mirás bien nos pasamos todo el tiempo:
* _jugando_ juegos ej. "consorcio y expensas", "pagar para comer", etc.
* _evaluando_ juegos ej. "¿compro bitcoin?"
* _ diseñando_ juegos ej. "podés elegir entre el plan miserable, el carísimo y el que yo quiero que elijas" o "vení a trabajar a mi empresa"
* _arreglando_ todos los juegos de arriba


## Necesitamos hacer juegos ¿Cómo?

* "de palabra", cuando organizamos, charlamos, etc.
* "de mesa", con papeles, tapitas, cartas, ...
* "con la compu", formularios, "motores"

Lo más importante:

* **probar cada un par de horas o menos**
* que sea fácil cambiar todo

_NOTAR_ que hacer esperar a la gente para que se resigne a lo que salió porque no podría seguir esperando es un _juego roto_ que se usa de base en muchísimas estafas ej. los "Planes de ahorro" lo mismo que hacerte dar pasos fáciles de dar pero casi imposibles de deshacer.

### NetLogo para explicar

http://www.netlogoweb.org/launch#http://www.netlogoweb.org/assets/modelslib/Sample%20Models/Social%20Science/Traffic%20Basic.nlogo

### Motores

* https://godotengine.org/ es software libre, web, android, escritorio, y tiene una comunidad numerosa y generosa.
* https://threejs.org/ es software libre web muy poderoso y cómodo, alcanza con tener un navegador para programar y jugar.